# Feature Extractor

This notebook extracts features (keypoints heatmaps) from dataset images.

The extracted features are saved in "<dataset_path>\extracted_features"

Starting with the module imports:

In [1]:
import glob #Search files
import json #JSON write
import pathlib #Path things
import os #Path things
import warnings #Modified call
from concurrent.futures import ThreadPoolExecutor #Threading
from types import MethodType #Modified call
from typing import Optional #Modified call

import numpy as np #Array operations
import imageio #EXR write
import tqdm #Progress bar
from rich.progress import track #Modified call
import mmpose #Feature extractor
from mmpose.apis import MMPoseInferencer #Feature extractor

Definition of execution variables:
- dataset_path: directory where the dataset is
- batch_size: inference batch size

In [2]:
dataset_path = os.environ["USERPROFILE"]+"\\AppData\\LocalLow\\DefaultCompany\\IA904-3D_Pose\\solo"
#dataset_path = "I:\\.shortcut-targets-by-id\\1S6q0nt4z5LYa-5VkpC8qxag2b_jjO6e9\\IA904\\Dataset"
batch_size = 512

Search for all dataset images paths:

In [3]:
search_pattern = dataset_path+"\\**\\*.png"
img_paths = glob.glob(search_pattern, recursive=True)

Initializes the feature extractor:

In [5]:
inferencer = MMPoseInferencer('human')

Loads checkpoint by http backend from path: https://download.openmmlab.com/mmpose/v1/projects/rtmposev1/rtmpose-m_simcc-body7_pt-body7_420e-256x192-e48f03d0_20230504.pth
06/18 15:40:17 - mmengine - WARNING - Failed to search registry with scope "mmpose" in the "function" registry tree. As a workaround, the current "function" registry in "mmengine" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmpose" is a correct scope, or whether the registry is initialized.
Loads checkpoint by http backend from path: https://download.openmmlab.com/mmpose/v1/projects/rtmposev1/rtmdet_m_8xb32-100e_coco-obj365-person-235e8209.pth
06/18 15:40:19 - mmengine - WARNING - Failed to search registry with scope "mmdet" in the "function" registry tree. As a workaround, the current "function" registry in "mmengine" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmdet" is a corre

## Inferencer custom \_\_call__

Because the MMPose doesn't provide the infered heatmaps, the `__call__` method needs to be modified for returning it (only works for images with only one person, it will generate images with all detected heatmaps for the same keypoint if more than one person is in the image):

In [6]:
#Base from MMPoseInferencer.__call__
def modified_call(
        self,
        inputs: mmpose.apis.inferencers.mmpose_inferencer.InputsType,
        return_datasamples: bool = False,
        batch_size: int = 1,
        out_dir: Optional[str] = None,
        **kwargs,
    ) -> dict:
        """Call the inferencer.

        Args:
            inputs (InputsType): Inputs for the inferencer.
            return_datasamples (bool): Whether to return results as
                :obj:`BaseDataElement`. Defaults to False.
            batch_size (int): Batch size. Defaults to 1.
            out_dir (str, optional): directory to save visualization
                results and predictions. Will be overoden if vis_out_dir or
                pred_out_dir are given. Defaults to None
            **kwargs: Key words arguments passed to :meth:`preprocess`,
                :meth:`forward`, :meth:`visualize` and :meth:`postprocess`.
                Each key in kwargs should be in the corresponding set of
                ``preprocess_kwargs``, ``forward_kwargs``,
                ``visualize_kwargs`` and ``postprocess_kwargs``.

        Returns:
            dict: Inference and visualization results.
        """
        if out_dir is not None:
            if 'vis_out_dir' not in kwargs:
                kwargs['vis_out_dir'] = f'{out_dir}/visualizations'
            if 'pred_out_dir' not in kwargs:
                kwargs['pred_out_dir'] = f'{out_dir}/predictions'
        
        kwargs = {
            key: value
            for key, value in kwargs.items()
            if key in set.union(self.inferencer.preprocess_kwargs,
                                self.inferencer.forward_kwargs,
                                self.inferencer.visualize_kwargs,
                                self.inferencer.postprocess_kwargs)
        }
        (
            preprocess_kwargs,
            forward_kwargs,
            visualize_kwargs,
            postprocess_kwargs,
        ) = self._dispatch_kwargs(**kwargs)

        self.inferencer.update_model_visualizer_settings(**kwargs)

        # preprocessing
        if isinstance(inputs, str) and inputs.startswith('webcam'):
            inputs = self.inferencer._get_webcam_inputs(inputs)
            batch_size = 1
            if not visualize_kwargs.get('show', False):
                warnings.warn('The display mode is closed when using webcam '
                              'input. It will be turned on automatically.')
            visualize_kwargs['show'] = True
        else:
            inputs = self.inferencer._inputs_to_list(inputs)
        self._video_input = self.inferencer._video_input
        if self._video_input:
            self.video_info = self.inferencer.video_info

        inputs = self.preprocess(
            inputs, batch_size=batch_size, **preprocess_kwargs)

        # forward
        if 'bbox_thr' in self.inferencer.forward_kwargs:
            forward_kwargs['bbox_thr'] = preprocess_kwargs.get('bbox_thr', -1)

        preds = []

        for proc_inputs, ori_inputs in (track(inputs, description='Inference')
                                        if self.show_progress else inputs):
            preds = self.forward(proc_inputs, **forward_kwargs)

            visualization = self.visualize(ori_inputs, preds,
                                           **visualize_kwargs)
            
            results = self.postprocess(
                preds,
                visualization,
                return_datasamples=return_datasamples,
                **postprocess_kwargs)
            
            #MODIFIED START-----------------------------------
            if kwargs["draw_heatmap"] is True:
                for batch_index in range(len(preds)):
                    pred = preds[batch_index]
                
                    for person_index in range(len(pred.pred_instances)):
                        person_pred = pred.pred_instances[person_index]
                        
                        results["predictions"][batch_index][person_index]["heatmaps"] = pred._pred_heatmaps.heatmaps
            #MODIFIED END-------------------------------------
            
            yield results

        if self._video_input:
            self._finalize_video_processing(
                postprocess_kwargs.get('pred_out_dir', ''))

In [7]:
inferencer.modified_call = MethodType(modified_call, inferencer)

## Inference and save

Now we just need to make the inference and save the extracted data.

Encoder function for serializing numpy arrays:

In [8]:
def encoder(obj):
    if isinstance(obj, np.integer):
            return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    
    return obj.__dict__

Creates the inference generator:

In [9]:
result_generator = inferencer.modified_call(img_paths, draw_heatmap=True, batch_size=batch_size)

Make the inferences and save the data:

In [10]:
executor = ThreadPoolExecutor()

def save_heatmaps(img_heatmaps, heatmaps_path):
    np.savez_compressed(heatmaps_path, img_heatmaps=img_heatmaps)

    #EXR export - More disk usage
    #for keypoint_index in range(len(img_heatmaps)): #keypoints in person
    #    keypoint_heatmap = img_heatmaps[keypoint_index]
    #
    #    heatmap_name = img_name+f"_heatmap{person_index}_{keypoint_index}"
    #    heatmap_name += ".exr"
    #    heatmap_path = os.path.join(extrated_features_dir, heatmap_name)
    #
    #    imageio.imwrite(heatmap_path, keypoint_heatmap)

def save_predictions(img_predictions, predictions_path):
    file = open(predictions_path, "w")
    json.dump(img_predictions, file, default=encoder)
    file.close()

In [11]:
futures = []

pbar = tqdm.tqdm(total=len(img_paths))

img_index = 0
for result in result_generator: #batchs
    predictions = result["predictions"]

    for batch_index in range(len(predictions)): #images in batch
        img_result = predictions[batch_index]

        img_path = img_paths[img_index] #Full image path
        img_dir = os.path.dirname(img_path) #Image directory
        img_name = os.path.splitext(os.path.basename(img_paths[img_index]))[0] #Image name without extension

        inside_dir = os.path.relpath(img_dir, dataset_path) #Path of image without the dataset_path prefix
        extrated_features_dir = os.path.join(dataset_path, "extracted_features", inside_dir) #Path of the image in the extracted_features dir
        pathlib.Path(extrated_features_dir).mkdir(parents=True, exist_ok=True) #Create image path if not exists

        if len(img_result) != 0:    
            for person_index in [0]:#range(len(img_result)): #persons in image
                img_heatmaps = predictions[batch_index][person_index]["heatmaps"]

                heatmaps_name = img_name+"_heatmaps"
                heatmaps_path = os.path.join(extrated_features_dir, heatmaps_name)

                future = executor.submit(save_heatmaps, img_heatmaps, heatmaps_path)
                futures.append(future)

                img_predictions =  predictions[batch_index][person_index].copy()
                del img_predictions["heatmaps"]

                predictions_name = img_name+"_kp2d_predictions.json"
                predictions_path = os.path.join(extrated_features_dir, predictions_name)

                future = executor.submit(save_predictions, img_predictions, predictions_path)
                futures.append(future)
                


        pbar.update(1)
        img_index += 1

for future in tqdm.tqdm(futures):
    future.result()

  0%|          | 0/19224 [00:00<?, ?it/s]c:\Python38\lib\site-packages\torch\functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3550.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
c:\Python38\lib\site-packages\mmpose\models\heads\coord_cls_heads\rtmcc_head.py:217: UserWarning: The predicted simcc values are normalized for visualization. This may cause discrepancy between the keypoint scores and the 1D heatmaps.
  warnings.warn('The predicted simcc values are normalized for '
100%|██████████| 38448/38448 [00:00<00:00, 270754.73it/s]


Inference time: 1 h 19 min 42,8 s